<a href="https://colab.research.google.com/github/aashitarakesh/Twitter_Analysis/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from google.colab import files
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [ ]:
# # loads data from google drive

# from google.colab import drive
# drive.mount("/content/gdrive")

# pd.read_csv('/content/gdrive/My Drive/UPenn Data Analytics/UPenn Data/')

In [ ]:
# # import data from local system

# from google.colab import files
# uploaded = files.upload()
# pd.read_csv("")

In [ ]:
# # import file from github

# from pyspark import SparkFiles

# url = 'https://raw.githubusercontent.com/aashitarakesh/Twitter_Analysis/main/sentiment_twitter_clean.csv'
# spark.sparkContext.addFile(url)
# df = spark.read.csv(SparkFiles.get("sentiment_twitter_clean.csv"), sep=",", header=True)

# # Show DataFrame
# df.show()


In [ ]:
# import file from github

from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/aashitarakesh/Twitter_Analysis/main/sentiment_twitter_clean.csv'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment_twitter_clean.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+-----+--------------------+-----+
|Index|              Tweets|Label|
+-----+--------------------+-----+
|  106|real good moment ...|    0|
|  217|       reading manga|    0|
|  288|need send em acco...|    0|
|  540|add myspace myspa...|    0|
|  624|sleepy good times...|    0|
|  701|nbn someone alrea...|    0|
|  808|23 24ï½c possible...|    0|
| 1193|nite twitterville...|    0|
| 1324|night darlin swee...|    0|
| 1332|good morning ever...|    0|
| 1368|finally created w...|    0|
| 1578|kisha cnt get u t...|    0|
| 1595|yes remember band...|    0|
| 1861|really love refle...|    0|
| 1889|ooo fantasy like ...|    0|
| 1899|probs sell nothin...|    0|
| 1919|quotnokla connect...|    0|
| 1992|stayed late start...|    0|
| 2097|read new job cong...|    0|
| 2126|havent able sleep...|    0|
+-----+--------------------+-----+
only showing top 20 rows



In [ ]:
# # Drop all rows with missing information
# df = df.dropna(how='any')


In [ ]:
# # Verify dropped rows
# df.count()
# df

DataFrame[Index: string, Tweets: string, Label: string]

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['Tweets']))
data_df.show()

+-----+--------------------+-----+------+
|Index|              Tweets|Label|length|
+-----+--------------------+-----+------+
|  106|real good moment ...|    0|    34|
|  217|       reading manga|    0|    13|
|  288|need send em acco...|    0|    92|
|  540|add myspace myspa...|    0|    33|
|  624|sleepy good times...|    0|    32|
|  701|nbn someone alrea...|    0|    54|
|  808|23 24ï½c possible...|    0|    28|
| 1193|nite twitterville...|    0|    30|
| 1324|night darlin swee...|    0|    25|
| 1332|good morning ever...|    0|    22|
| 1368|finally created w...|    0|    75|
| 1578|kisha cnt get u t...|    0|    45|
| 1595|yes remember band...|    0|    38|
| 1861|really love refle...|    0|    31|
| 1889|ooo fantasy like ...|    0|    37|
| 1899|probs sell nothin...|    0|    50|
| 1919|quotnokla connect...|    0|    31|
| 1992|stayed late start...|    0|    43|
| 2097|read new job cong...|    0|    44|
| 2126|havent able sleep...|    0|    51|
+-----+--------------------+-----+

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='Label',outputCol='label')
tokenizer = Tokenizer(inputCol="Tweets", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)


In [ ]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[76764,11...|
|  0.0|(262145,[41314,12...|
|  0.0|(262145,[942,2032...|
|  0.0|(262145,[48063,98...|
|  0.0|(262145,[73853,11...|
|  0.0|(262145,[2306,178...|
|  0.0|(262145,[22346,44...|
|  0.0|(262145,[62166,93...|
|  0.0|(262145,[88594,15...|
|  0.0|(262145,[113432,1...|
|  0.0|(262145,[2306,762...|
|  0.0|(262145,[18176,51...|
|  0.0|(262145,[14118,42...|
|  0.0|(262145,[186480,2...|
|  0.0|(262145,[13823,12...|
|  0.0|(262145,[14118,16...|
|  0.0|(262145,[93523,10...|
|  0.0|(262145,[16004,68...|
|  0.0|(262145,[10345,53...|
|  0.0|(262145,[9657,506...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
cleaned.printSchema()

root
 |-- Index: string (nullable = true)
 |-- Tweets: string (nullable = true)
 |-- label: double (nullable = false)
 |-- length: integer (nullable = true)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hash_token: vector (nullable = true)
 |-- idf_token: vector (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
cleaned.show()
cleaned.count()

+-----+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Index|              Tweets|label|length|          token_text|         stop_tokens|          hash_token|           idf_token|            features|
+-----+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  106|real good moment ...|  0.0|    34|[real, good, mome...|[real, good, mome...|(262144,[76764,11...|(262144,[76764,11...|(262145,[76764,11...|
|  217|       reading manga|  0.0|    13|    [reading, manga]|    [reading, manga]|(262144,[41314,12...|(262144,[41314,12...|(262145,[41314,12...|
|  288|need send em acco...|  0.0|    92|[need, send, em, ...|[need, send, em, ...|(262144,[942,2032...|(262144,[942,2032...|(262145,[942,2032...|
|  540|add myspace myspa...|  0.0|    33|[add, myspace, my...|[add, myspace, my...|(262144,[48063,98...|(262144,[48063

10283

In [ ]:
count = cleaned.count()

# export dataframe to a csv

# import numpy as np
# import pandas as pd

# # Enable Arrow-based columnar data transfers
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# pandas_df = cleaned.select("*").toPandas()


In [ ]:
# pandas_df.to_csv('data.csv')

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()
# test_results.show(count)

+------+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| Index|              Tweets|label|length|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+------+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|100077|       dyededed hair|  0.0|    13|    [dyededed, hair]|    [dyededed, hair]|(262144,[126768,1...|(262144,[126768,1...|(262145,[126768,1...|[-174.52756472984...|[0.99942235398681...|       0.0|
|101065|good morning haha...|  0.0|    79|[good, morning, h...|[good, morning, h...|(262144,[24657,53...|(262144,[24657,53...|(262145,[24657,53...|[-833.54066304901...|[1.0,4.4906118047...|       

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.907054


In [ ]:
# export dataframe to a csv

import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

results = test_results.select("*").toPandas()

results.to_csv('nb_results.csv')


/content/spark-3.0.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
